In [1]:
!git clone https://github.com/Pyroxene889/Store-Sales

Cloning into 'Store-Sales'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 39 (delta 15), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (39/39), 89.83 KiB | 1.80 MiB/s, done.


In [2]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

# Data Cleaning

## Data Collecting

In [4]:
od.download(
	"https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data?select=holidays_events.csv", force = True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ikramlaaouane
Your Kaggle Key: ··········


100%|██████████| 21.4M/21.4M [00:01<00:00, 11.2MB/s]



Extracting archive ./store-sales-time-series-forecasting/store-sales-time-series-forecasting.zip to ./store-sales-time-series-forecasting


In [104]:
df_train1 = pd.read_csv('/content/store-sales-time-series-forecasting/train.csv')

In [142]:
df_train1.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [124]:
df_test1 = pd.read_csv('/content/store-sales-time-series-forecasting/test.csv')

In [143]:
df_test1.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [125]:
df_stores = pd.read_csv('/content/store-sales-time-series-forecasting/stores.csv')

In [126]:
df_holidays = pd.read_csv('/content/store-sales-time-series-forecasting/holidays_events.csv')
df_holidays["type"].value_counts().index

Index(['Holiday', 'Event', 'Additional', 'Transfer', 'Bridge', 'Work Day'], dtype='object')

In [127]:
df_oil = pd.read_csv('/content/store-sales-time-series-forecasting/oil.csv')
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [128]:
df_transactions = pd.read_csv('/content/store-sales-time-series-forecasting/transactions.csv')

### Clean table oil

In [129]:
def fillna_oil(df):
  df2=df
  df2['oil2']=df2['dcoilwtico'].shift(1)
  df2['oil3']=df2['dcoilwtico'].shift(-1)

  df2.loc[df2['dcoilwtico'].isnull(),'dcoilwtico']=df2['oil2']
  df2.loc[df2['dcoilwtico'].isnull(),'dcoilwtico']=df2['oil3']

  df2.drop(['oil2', 'oil3'], axis=1,inplace=True)
  df2.head()

  df2.head(20)
  return df2

In [130]:
# Clean la table oil
df_oil = fillna_oil(df_oil)

In [131]:
df_oil.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [132]:
df_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1218 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


### Merge tous la data en une seule DataFrame

In [155]:
def Merge_Data(df, p=0):
  ## Merge train_data avec stores_data
  #df = df.set_index('store_nbr').join(df_stores.set_index('store_nbr'))
  df = df.join(df_stores.set_index('store_nbr'), on='store_nbr')
  ## Merge la data avec holidays_data
  df = df.set_index('date').join(df_holidays.set_index('date'), lsuffix='_store', rsuffix='_holiday')
  ## Merge la data avec oil_data
  df = df.join(df_oil.set_index('date'), on='date')
  ## Merge la data avec transactions pour le train
  if p==0:
    df = pd.merge(df, df_transactions, on=['date', 'store_nbr'])
  return df


Merge Data Train

In [156]:
df_train_merge = Merge_Data(df_train1)
df_train_merge.head()

,date,id,store_nbr,family,sales,onpromotion,city,state,type_store,cluster,type_holiday,locale,locale_name,description,transferred,dcoilwtico,transactions
0,2013-01-01,561,25,AUTOMOTIVE,0.0,0,Salinas,Santa Elena,D,1,Holiday,National,Ecuador,Primer dia del ano,False,93.14,770
1,2013-01-01,562,25,BABY CARE,0.0,0,Salinas,Santa Elena,D,1,Holiday,National,Ecuador,Primer dia del ano,False,93.14,770
2,2013-01-01,563,25,BEAUTY,2.0,0,Salinas,Santa Elena,D,1,Holiday,National,Ecuador,Primer dia del ano,False,93.14,770
3,2013-01-01,564,25,BEVERAGES,810.0,0,Salinas,Santa Elena,D,1,Holiday,National,Ecuador,Primer dia del ano,False,93.14,770
4,2013-01-01,565,25,BOOKS,0.0,0,Salinas,Santa Elena,D,1,Holiday,National,Ecuador,Primer dia del ano,False,93.14,770


Merge Test Data

In [157]:
df_test_merge = Merge_Data(df_test1,1)
df_test_merge.head()

,id,store_nbr,family,onpromotion,city,state,type_store,cluster,type_holiday,locale,locale_name,description,transferred,dcoilwtico
date,,,,,,,,,,,,,,
2017-08-16,3000888,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
2017-08-16,3000889,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
2017-08-16,3000890,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
2017-08-16,3000891,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
2017-08-16,3000892,1,BOOKS,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8


### Data Cleaning

In [136]:
df_train_merge.isnull().sum()

date                  0
id                    0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type_store            0
cluster               0
type_holiday    2346168
locale          2346168
locale_name     2346168
description     2346168
transferred     2346168
dcoilwtico       807246
transactions          0
dtype: int64

In [137]:
def Cleaning(df):

  # Transformer les jours fériers qui ont été transférré en un autre jour en jour normal "work Day"
  df.loc[df['transferred'] == True, 'type_holiday'] = 'Work Day'

  #Supprimer la colonne "locale_name", "description" et "transferred", puisqu'on n'en a pas besoin
  df = df.drop(['id'], axis = 1)
  df = df.drop(['locale_name'], axis = 1)
  df = df.drop(['description'], axis = 1)
  df = df.drop(['transferred'], axis = 1)
  # Remplacer les valeurs NaN de la colonne "type_holiday" avec "work"
  df["type_holiday"].fillna("work", inplace = True)
  # Remplacer les valeurs NaN de la colonne "locale" avec "National"
  df["locale"].fillna("National", inplace = True)

  
  df['oil2']=df['dcoilwtico'].shift(1)
  df['oil3']=df['dcoilwtico'].shift(-1)

  df.loc[df['dcoilwtico'].isnull(),'dcoilwtico']=df['oil2']
  df.loc[df['dcoilwtico'].isnull(),'dcoilwtico']=df['oil3']

  df.drop(['oil2', 'oil3'], axis=1,inplace=True)
  


  return df


In [138]:
df_train_clean = Cleaning(df_train_merge)
df_train_clean.isnull().sum()

date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
city                 0
state                0
type_store           0
cluster              0
type_holiday         0
locale               0
dcoilwtico      806764
transactions         0
dtype: int64

In [139]:
df_train_clean.loc[df_train_clean['dcoilwtico'].isnull()]

,date,store_nbr,family,sales,onpromotion,city,state,type_store,cluster,type_holiday,locale,dcoilwtico,transactions
4588,2013-01-05,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,Work Day,National,NaN,1509
4589,2013-01-05,1,BEAUTY,3.000,0,Quito,Pichincha,D,13,Work Day,National,NaN,1509
4590,2013-01-05,1,BEVERAGES,1160.000,0,Quito,Pichincha,D,13,Work Day,National,NaN,1509
4591,2013-01-05,1,BOOKS,0.000,0,Quito,Pichincha,D,13,Work Day,National,NaN,1509
4592,2013-01-05,1,BREAD/BAKERY,301.057,0,Quito,Pichincha,D,13,Work Day,National,NaN,1509
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2801661,2017-08-13,9,PLAYERS AND ELECTRONICS,9.000,0,Quito,Pichincha,B,6,work,National,NaN,1745
2801662,2017-08-13,9,POULTRY,412.458,0,Quito,Pichincha,B,6,work,National,NaN,1745
2801663,2017-08-13,9,PREPARED FOODS,105.169,1,Quito,Pichincha,B,6,work,National,NaN,1745
2801664,2017-08-13,9,PRODUCE,1693.607,7,Quito,Pichincha,B,6,work,National,NaN,1745
